In [1]:
import cv2 
import numpy as np 
from matplotlib import pyplot as plt

In [ ]:
def findMaxContour(contours):
    max_i = 0 #max contouru veren indis
    max_area = 0
    
    for i in  range(len(contours)): # alanları tek tek karsılastırıp max olanı bulucaz
        
        area_face = cv2.contoursArea(contours[i]) # gezilen contoursun alanları degişkene aktarılır
        
        if max_area < area_face:  # alınan contour  max_area dan büyükse
            max_area = area_face # max_area değişkenine bu contour atanır
            max_i = i # büyük olan contour atanır 
            
        try:
            
            cnt = contours[max_i] # büyük olan contour atanır
            
        except: #eğer try bloğu gerceklesmesse  except kısmı gercekleşecektir
            contours = [0]
            c = contours[0]
            
    return cnt
        

In [ ]:
vid = cv2.VideoCapture(0)

while 1:
    
    ret , frame = vid.read()
    frame = cv2.flip(frame , 1)
    
    roi = frame[50:250 , 200:400]
    cv2.rectangle(frame , (250,50) , (450,250) , (0,0,255) ,0) # frame[y1:y2,x1:x2]   
    
    # yüz kısmını diğer görüntülerden ayırmak için hsv'ye ceviricez
    hsv = cv2.cvtColor(roi , cv2.COLOR_BGR2HSV)
    
    # yüz için belirlenen hsv  aralığı girilir
    lower_color = np.array([0,45,79],dtype = np.uint8)
    upper_color = np.array([17,255,255] , dtype=np.uint8)
    
    mask = cv2.inRange(hsv  ,lower_color , upper_color)
    
    # görüntümüzün içindeki benekleri kernel ile yok edeceğiz
    kernel = np.ones((5,5) , np.uint8)
    mask = cv2.dilate(mask , kernel , iteration = 1)
    mask =  cv2.medianBlur(mask ,5) # median blurlama
    
    contours ,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # max contour değeri cekilir gürültüleri contour olarak almamak için
        try :
            c = findMaxContour(contours)
            
            # contour'un en uc  noktaları bulunur
            extLeft = tuple(c[:,:,0].argmin()[0])
            extRight = tuple(c[:,:,0].argmax()[0])
            extTop = tuple(c[:,:,1].argmin()[0])
            extBot = tuple(c[:,:,1].argmax()[0])
            
            cv2.circle(roi, extLeft,(0,255,0),2)
            cv2.circle(roi, extRight,(0,255,0),2)
            cv2.circle(roi, extTop,(0,255,0),2)
            cv2.circle(roi, extBot,(0,255,0),2)
            
        except:
            pass    
    
    cv2.imshow("frame",frame)
    if cv2.waitKey(0) & 0xFF == ord("q"):
        break
vid.release()
cv2.destroyAllWindows()